<a href="https://colab.research.google.com/github/vitorsr/ccd/blob/master/maps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prophet
## Quick Start

Source: https://facebook.github.io/prophet/docs/quick_start.html

In [4]:
!pip install -q geopandas

     |████████████████████████████████| 921kB 2.8MB/s 
     |████████████████████████████████| 11.9MB 193kB/s 
     |████████████████████████████████| 10.1MB 47.8MB/s 


In [0]:
import pandas as pd
import geopandas as gp
import matplotlib.pyplot as plt
import plotly.express as px
from google.colab import files
from fbprophet import Prophet

In [0]:
!unzip ccd_2019.zip

unzip:  cannot find or open ccd_2019.zip, ccd_2019.zip.zip or ccd_2019.zip.ZIP.


In [0]:
stations_meta = pd.read_csv("https://media.githubusercontent.com/media/vitorsr/ccd/master/data/bdmep_meta.csv?token=AFPQLZC3565PRPVW4B6VHPK5WIBZQ")

In [0]:
stations_meta.shape

In [0]:
stations_meta.columns

In [0]:
stations_meta.head()

In [0]:
stations_per_uf = stations_meta.groupby('uf').size().reset_index(name='stations_per_uf')
stations_per_uf_sorted_desc = stations_per_uf.sort_values('stations_per_uf', ascending=False)
stations_per_uf_sorted_desc.head()

In [0]:
inmetr = pd.read_csv("https://media.githubusercontent.com/media/vitorsr/ccd/master/data/inmetr.csv?token=AFPQLZE3FSY4QGCXIW4JTAC5WICC4")

In [8]:
inmetr.shape

(14528638, 17)

In [0]:
inmetr.columns

In [0]:
inmetr.head()

In [0]:
df = pd.merge(stations_meta, inmetr, on='id')
df

In [0]:
df_by_station = df.groupby(['lon', 'lat', 'name'])
df_by_station_count_des = df_by_station.size().reset_index(name='num_of_measurements_by_station').sort_values('num_of_measurements_by_station', ascending=False)
df_by_station_count_des

In [0]:
fig = px.scatter_geo(data_frame=df_by_station_count_des.head(3), scope='south america',lat='lat',lon='lon',
                     size='num_of_measurements_by_station', color='name')
fig.update_layout(
        title_text = '317')
fig.show()

In [0]:
df_by_uf = df.groupby('uf')
df_by_uf.size().reset_index(name='num_of_measurements_by_uf')

In [0]:
df_by_uf.get_group('RN')['name'].drop_duplicates()

In [0]:
nan_prec = 100.0 * ((len(df) - df['prec'].count()) / len(df))
nan_tair = 100.0 * ((len(df) - df['tair'].count()) / len(df))
nan_tw = 100.0 * ((len(df) - df['tw'].count()) / len(df))
nan_tmax = 100.0 * ((len(df) - df['tmax'].count()) / len(df))
nan_tmin = 100.0 * ((len(df) - df['tmin'].count()) / len(df))
nan_urmax = 100.0 * ((len(df) - df['urmax'].count()) / len(df))
nan_patm = 100.0 * ((len(df) - df['patm'].count()) / len(df))
nan_pnmm = 100.0 * ((len(df) - df['pnmm'].count()) / len(df))
nan_wd = 100.0 * ((len(df) - df['wd'].count()) / len(df))
nan_wsmax = 100.0 * ((len(df) - df['wsmax'].count()) / len(df))
nan_n = 100.0 * ((len(df) - df['n'].count()) / len(df))
nan_cc = 100.0 * ((len(df) - df['cc'].count()) / len(df))
nan_evap = 100.0 * ((len(df) - df['evap'].count()) / len(df))
nan_ur = 100.0 * ((len(df) - df['ur'].count()) / len(df))
nan_ws = 100.0 * ((len(df) - df['ws'].count()) / len(df))

print("Precipitação (%): ", nan_prec.round())
print("Temperatura do Ar (%): ", nan_tair.round())
print("Temperatura de Bulbo Úmido (%): ", nan_tw.round())
print("Temperatura Máxima do Ar (%): ", nan_tmax.round())
print("Temperatura Mínima do Ar (%): ", nan_tmin.round())
print("Umidade Relativa Máxima (%): ", nan_urmax.round())
print("Pressão Atmosférica (%): ", nan_patm.round())
print("Pressão Atmosférica Média ao Nível do Mar (%): ", nan_pnmm.round())
print("Direção do Vento (%): ", nan_wd.round())
print("Rajadas de Vento (%): ", nan_wsmax.round())
print("Horas de Sol (%): ", nan_n.round())
print("Cobertura de Nuvens (%): ", nan_cc.round())
print("Evaporação (%): ", nan_evap.round())
print("Umidade Relativa (%): ", nan_ur.round())
print("Velocidade do Vento (%): ", nan_ws.round())

In [0]:
df = pd.DataFrame(
    {'City': ['Buenos Aires', 'Brasilia', 'Santiago', 'Bogota', 'Caracas'],
     'Country': ['Argentina', 'Brazil', 'Chile', 'Colombia', 'Venezuela'],
     'Latitude': [-34.58, -15.78, -33.45, 4.60, 10.48],
     'Longitude': [-58.66, -47.91, -70.66, -74.08, -66.86]})

gdf = gp.GeoDataFrame(
    df, geometry=gp.points_from_xy(df.Longitude, df.Latitude))

print(gdf.head())

In [0]:
world = gp.read_file(gp.datasets.get_path('naturalearth_lowres'))

# We restrict to Brazil.
ax = world[world.name == 'Brazil'].plot(
    color='blue', edgecolor='black')

# We can now plot our ``GeoDataFrame``.
gdf.plot(ax=ax, color='red')

plt.show()

In [0]:

df = pd.read_csv("https://raw.githubusercontent.com/facebook/prophet/master/examples/example_wp_log_peyton_manning.csv")
df.head()

In [0]:
m = Prophet()
m.fit(df)

In [0]:
future = m.make_future_dataframe(periods=365)
future.tail()

In [0]:
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

In [0]:
fig1 = m.plot(forecast)

In [0]:
fig2 = m.plot_components(forecast)